In [26]:
import pandas as pd

# Components

### Incidence by region and age groups

In [27]:
incidence = pd.read_csv("incidence.csv")

incidence.head()

,year,category,region,age_group,incidence
0,2010,Злоякiснi новоутворення-всього C00-C97...,Автономна pеспубліка Крим,до 1 р.,0
1,2010,Злоякiснi новоутворення-всього C00-C97...,Автономна pеспубліка Крим,1-4 р.,6
2,2010,Злоякiснi новоутворення-всього C00-C97...,Автономна pеспубліка Крим,5-9 р.,4
3,2010,Злоякiснi новоутворення-всього C00-C97...,Автономна pеспубліка Крим,10-14р,6
4,2010,Злоякiснi новоутворення-всього C00-C97...,Автономна pеспубліка Крим,15-19р,6


### Incidence by region and stage

In [28]:
stage_incidence = pd.read_csv("stage_date.csv")

stage_incidence.head()

,year,region,mtumors,syncmtumors,insitu,ncervix
9,2010,Автономна pеспубліка Крим,585,165,39,25
10,2010,Вінницька,356,112,113,103
11,2010,Волинська,177,50,81,77
12,2010,Дніпропетровська,767,212,174,150
13,2010,Донецька,906,234,57,24


#### F470100

In [29]:
f47_0100_df = pd.read_csv("F470100.csv")
f47_0100_df['category'] = f47_0100_df.category.str.replace(' ', '')
aggregated_f47_0100_df = f47_0100_df[f47_0100_df['category'] == "ВСЬОГО"]
aggregated_f47_0100_df = aggregated_f47_0100_df.drop(columns=['category'])

aggregated_f47_0100_df.head()

,year,region,nhospotal,nbeds,ybeds,nill,nvillage_ill,bed_days,dvisits,hvisits,ndoctors,nnursing
702,2010,Автономна pеспубліка Крим,75,15098,15134,396822,126271,4940506,10105234,691176,5663.7,11139.75
703,2010,Вінницька,109,12441,12423,368335,190347,4160024,10388187,767212,4461.25,9470.5
704,2010,Волинська,52,7653,7650,229994,112263,2633088,5988156,340466,2757,5835.5
705,2010,Дніпропетровська,155,29713,29892,805622,129025,10084180,24579242,2029582,10911.5,22011.75
706,2010,Донецька,173,32134,32193,893383,73028,10803773,36517794,2850518,13726,30085


#### F471700 - equipment

In [30]:
f_47_1700_df = pd.read_csv('F471700.csv')

categories_variables = {
    "nx_ray": "Чис.закл.,які мають:рентг.від.(каб)",
    "nflurography": "флюорографiчнi вiддiлення (кабiнети)",
    "nradiology": "радiологiчнi (променевої терапiї)",
    "nradlab": "лабораторії радіоізотопної діагност.",
    "nсt": "комп'ютерної томографії",
    "ncardiogram": "каб.електрокардіог.та функц.діагност.",
    "ndiaglab": "клініко-діагностичні лабораторії",
    "nbacter": "бактеріологічні лабораторії",
    "nbiochem": "біохімічні лабораторії",
    "ncyto": "цитологічні лабораторії",
    "nimun": "імунологічні лабораторії",
    "nphysic": "фізіотерапевтичні відділення (каб.)",
    "nendoscop": "ендоскопічні відділення (кабінети)",
    "nultrasound": "ультразвукової діагностики",
    "ndialysis": "відділення нефрології та діалізу"
}

equipment_dfs = []
for column_name, category in categories_variables.items():
    df = f_47_1700_df[f_47_1700_df['category'] == category][['year', 'region', 'total']]
    df = df.rename(columns={"total": column_name})
    if len(df) == 0:
        raise Exception("inconsistent")
    equipment_dfs.append(df)

equipment_df = equipment_dfs[0]
for df in equipment_dfs[1:]:
    equipment_df = pd.merge(equipment_df, df, on=["year", "region"], how="outer")

equipment_df.head()

,year,region,nx_ray,nflurography,nradiology,nradlab,nсt,ncardiogram,ndiaglab,nbacter,nbiochem,ncyto,nimun,nphysic,nendoscop,nultrasound,ndialysis
0,2008,Івано-Франківська,72,48,2,0,3,137,208,6,0,3,1,208,35,52,5
1,2008,Автономна pеспубліка Крим,97,56,8,2,5,111,216,13,6,1,3,241,33,54,3
2,2008,Волинська,47,31,1,1,3,122,137,3,1,2,0,139,26,36,2
3,2008,Вінницька,90,61,4,1,4,241,267,6,1,1,0,275,39,53,1
4,2008,Дніпропетровська,163,117,7,3,6,265,262,7,2,3,5,340,90,109,2


### Air pollution

In [31]:
air_pollution_df = pd.read_csv("air_polution.csv")
air_pollution_df = air_pollution_df.rename(columns={'Region': 'region'})

air_pollution_df = air_pollution_df[air_pollution_df['year'] >= 2008]
air_pollution_df = air_pollution_df.drop(columns=["Unnamed: 0"])
air_pollution_df.head()

,region,year,air_pollution
450,Автономна Республіка Крим,2008,31.1
451,Вінницька,2008,130.3
452,Волинська,2008,10.0
453,Дніпропетровська,2008,952.3
454,Донецька,2008,1533.4


### Water pollution

In [32]:
water_pollution = pd.read_csv("merged_dumps.csv")
water_pollution = water_pollution.rename(columns={'Region': 'region'})

water_pollution = water_pollution[water_pollution['year'] >= 2008]

water_pollution.head()

,region,year,polluted_dumps,not_cleaned_dumps,dumps_not_cleaned_enough,num_clearing_plants
450,Автономна Республіка Крим,2008,71,13,NaN,270
451,Вінницька,2008,2,0,NaN,95
452,Волинська,2008,4,1,NaN,76
453,Дніпропетровська,2008,613,142,NaN,931
454,Донецька,2008,615,65,NaN,1577


### GDP

In [33]:
gdp = pd.read_csv("gdp.csv")
gdp = gdp.rename(columns={"Region": "region"})

gdp = gdp[gdp['year'] >= 2008]

gdp.head()

,region,year,gdp
100,Автономна\r\n Республіка Крим,2008,13898
101,Вінницька,2008,12061
102,Волинська,2008,12340
103,Дніпропетровська,2008,30918
104,Донецька,2008,26028


### CPI

In [34]:
cpi = pd.read_csv("inflation.csv", sep=r"\s+")
cpi.cpi = cpi.cpi.map(lambda x: float(x.replace(',', '.')))

cpi = cpi[cpi['year'] >= 2008]

cpi.head()

,year,cpi
8,2008,122.3
9,2009,112.3
10,2010,109.1
11,2011,104.6
12,2012,99.8


### Population

In [39]:
population = pd.read_csv("population.csv")
population = population.rename(columns={"Територіальний розріз": "region", "Період": "year", "Значення cпостереження": "population"})

population = population[population["year"] >= 2008]

population.head()

,region,year,population
19,Автономна Республіка Крим,2008,1971072.0
20,Автономна Республіка Крим,2009,1967260.0
21,Автономна Республіка Крим,2010,1965305.0
22,Автономна Республіка Крим,2011,1963514.0
23,Автономна Республіка Крим,2012,1963008.0


# Datasets

### Age Group - dataset

In [42]:
age_group_incidence = pd.merge(incidence, aggregated_f47_0100_df, on=['year', 'region'], how='outer')
age_group_incidence = pd.merge(age_group_incidence, equipment_df, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, gdp, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, air_pollution_df, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, water_pollution, on=["year", "region"], how="outer")
age_group_incidence = pd.merge(age_group_incidence, cpi, on="year", how="outer")
age_group_incidence = pd.merge(age_group_incidence, population, on=['year', 'region'], how='outer')

age_group_incidence.to_csv("age_group_incidence.csv", index=False)

In [43]:
age_group_incidence.head()

,year,category,region,age_group,incidence,nhospotal,nbeds,ybeds,nill,nvillage_ill,...,nultrasound,ndialysis,gdp,air_pollution,polluted_dumps,not_cleaned_dumps,dumps_not_cleaned_enough,num_clearing_plants,cpi,population
0,2008,Злоякiснi новоутворення-всього C00-C97...,Івано-Франківська,до 1 р.,1.0,63.0,10786.0,10786,302630.0,172379.0,...,52.0,5.0,12940,241.7,19,0,NaN,131,122.3,1382603.0
1,2008,Злоякiснi новоутворення-всього C00-C97...,Івано-Франківська,до 1 р.,1.0,63.0,10786.0,10786,302630.0,172379.0,...,52.0,5.0,12940,241.7,19,0,NaN,131,122.3,1382603.0
2,2008,Злоякiснi новоутворення-всього C00-C97...,Івано-Франківська,до 1 р.,1.0,63.0,10786.0,10786,302630.0,172379.0,...,52.0,5.0,12940,241.7,19,0,NaN,131,122.3,1382603.0
3,2008,Злоякiснi новоутворення-всього C00-C97...,Івано-Франківська,1-4 р.,0.0,63.0,10786.0,10786,302630.0,172379.0,...,52.0,5.0,12940,241.7,19,0,NaN,131,122.3,1382603.0
4,2008,Злоякiснi новоутворення-всього C00-C97...,Івано-Франківська,1-4 р.,0.0,63.0,10786.0,10786,302630.0,172379.0,...,52.0,5.0,12940,241.7,19,0,NaN,131,122.3,1382603.0


### Stage dataset

In [37]:
stage_dataset = pd.merge(stage_incidence, aggregated_f47_0100_df, on=['year', 'region'], how='outer')
stage_dataset = pd.merge(stage_dataset, equipment_df, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, gdp, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, air_pollution_df, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, water_pollution, on=["year", "region"], how="outer")
stage_dataset = pd.merge(stage_dataset, cpi, on="year", how="outer")
stage_dataset = pd.merge(stage_dataset, population, on=['year', 'region'], how='outer')

stage_dataset.to_csv("stage_incidence.csv")